In [1]:
import numpy as np
import open3d as o3d
import copy

In [2]:
from readData import *
from showFig import *
from showCloud import *
from warp import *

In [3]:
points0, mat0 = getPoints("..\dataset\scan1.npy", prange=(200, 150, (-100, -30)), skip=20)
points1, mat1 = getPoints("..\dataset\scan2.npy", prange=(200, 150, (-100, -30)), skip=20)

read in ..\dataset\scan1.npy
x range: -197 ~ 198
y range: -158 ~ 145
z range: -69 ~ 0
read in ..\dataset\scan2.npy
x range: -197 ~ 1326
y range: -1076 ~ 145
z range: -10150 ~ 365


In [5]:
def draw_transform(src, tgt, T):
    srct = copy.deepcopy(src)
    tgtt = copy.deepcopy(tgt)
    srct.paint_uniform_color([1, 0.5, 0])
    tgtt.paint_uniform_color([0, 0.5, 0.9])
    srct.transform(T)
    o3d.visualization.draw_geometries([srct, tgtt])
    return src.transform(T)

In [6]:
def preprocess(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
    )

    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100)
    )

    return pcd_down, pcd_fpfh 

In [7]:
voxel_size = 0.01
src = o3d.geometry.PointCloud()
src.points = o3d.utility.Vector3dVector(points0)
tgt = o3d.geometry.PointCloud()
tgt.points = o3d.utility.Vector3dVector(points1)
# o3d.visualization.draw_geometries([src, tgt])
srcd, srcf = preprocess(src, voxel_size)
tgtd, tgtf = preprocess(tgt, voxel_size)

In [9]:
fig = dynVisual([np.asarray(src.points), np.asarray(tgt.points)], ["src", "tgt"], 1, 4)
fig.write_html("cloudpoint.html")

In [10]:
result = o3d.pipelines.registration.registration_fast_based_on_feature_matching(
    srcd, tgtd,
    srcf, tgtf
)
draw_transform(srcd, tgtd, result.transformation)

PointCloud with 35445 points.